In [53]:
#Скачиваем необходимые модули Питона
!pip install fake-useragent
!pip install tqdm

In [95]:
#Собственно выкачивание файлов из Интернета
import requests

#Модуль для работы с HTML-файлами
from bs4 import BeautifulSoup

#Работа с таблицами — понадобится, чтобы хранить наш корпус
import pandas as pd

#Поможет замаскироваться под обычного пользователя в Интернете
from fake_useragent import UserAgent 

#Переменные, нужные, чтобы маскироваться под обычного пользователя Интернета
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

In [96]:
#Кортеж из трёх новостей
news_links = ('https://www.zvportal.ru/index.php/2010-11-20-05-58-23-16/34-2010-11-20-06-29-25/15650-vremya-podumat', 
              'https://www.zvportal.ru/index.php/2010-11-20-05-58-23-27/72-2010-11-20-06-48-43/15643-byudzhet-uvelichen', 
              "https://www.zvportal.ru/index.php/2010-11-20-05-58-23-13/17-2010-11-20-06-11-2816/15662-musor-ne-projdjot")

#Создаём список из всех ответов сайта
requests_list = []
for link in news_links:
    req = requests.get(link, headers=headers)
    requests_list.append(req)
    
requests_list

[<Response [200]>, <Response [200]>, <Response [200]>]

In [97]:
#Создаём список из "супов"
bodys_list = []

for req in requests_list:
    #Просим модуль проанализировать наш файл
    soup = BeautifulSoup(req.text, 'html.parser')
    #Ищем
    body_soup = soup.find("div", class_="item-page")
    bodys_list.append(body_soup)

bodys_list[0]

<div class="item-page"><article class="art-post"><h2 class="art-postheader">Время подумать</h2><div class="art-postheadericons art-metadata-icons"><span class="art-postdateicon">Создано 15.03.2023 05:21</span> | <span class="art-postdateicon">Обновлено 15.03.2023 05:25</span> | <span class="art-postauthoricon">Автор: Марина Остащук</span> | <a href="/index.php/2010-11-20-05-58-23-16/34-2010-11-20-06-29-25/15650-vremya-podumat?tmpl=component&amp;print=1&amp;layout=default&amp;page=" onclick="window.open(this.href,'win2','status=no,toolbar=no,scrollbars=yes,titlebar=no,menubar=no,resizable=yes,width=640,height=480,directories=no,location=no'); return false;" rel="nofollow" title="Печать"><img alt="Печать" src="/templates/zvportal_old_fashion/images/system/printButton.png"/></a> | Просмотров: 39</div><div class="art-postcontent clearfix"><div class="art-article"><p>«Мы на месяц закрываем амурские дороги для большегрузов. Это необходимо, чтобы избежать разрушения трасс. Весной, когда грунт

In [98]:
#Создаём список текстов
textes_news = []

for body in bodys_list:
    cher_news = ""
    for p in body.find_all("p"): 
        cher_news += p.text.strip() 
    textes_news.append(cher_news)
    
textes_news[0]

'«Мы на месяц закрываем амурские дороги для большегрузов. Это необходимо, чтобы избежать разрушения трасс. Весной, когда грунт под дорогами оттаивает, тяжёлые грузовики могут нанести им большой вред. С 1 апреля ограничения начнут действовать в южных районах, а на севере – с 15 апреля.У предприятий и грузоперевозчиков есть время продумать свои маршруты, чтобы избежать простоев на производстве. Ограничения не затронут пассажирские автобусы и сельхозтехнику, которая задействована на весенне-полевых работах», – сказал губернатор Приамурья Василий Орлов.Министерством транспорта и дорожного хозяйства Амурской области планируется введение ограничений движения транспортных средств общей массой более 12 тонн на дорогах регионального значения в период с 15 апреля по 14 мая 2023 года: Заречное – Сиан (от 0 до 41,7 км), Зея – Золотая Гора (от 0 до 83,7 км), Зея – Снежногорский (от 0 до 20 км), Зея – Тыгда (от 0 до 102,8 км).Администрация Зейского района планирует введение аналогичных ограничений н

In [99]:
#Создаём список заголовков
headers_news = []

for body in bodys_list:
    head_news = body.find("h2").text.strip()
    headers_news.append(head_news)
    
headers_news

['Время подумать', 'Бюджет увеличен', 'Мусор не пройдёт!']

In [100]:
#Создаём список дат
dates_news = []

for body in bodys_list:
    data_news = body.find("span", class_ = "art-postdateicon").text.strip()
    dates_news.append(data_news[8:])
    
dates_news


['15.03.2023 05:21', '10.03.2023 01:17', '17.03.2023 06:46']

In [101]:
#Добавляем в особую табличку тексты

corpus = pd.DataFrame({'title': headers_news,
                       'date': dates_news,
                       'text': textes_news,  
                       'link': news_links})
corpus

,title,date,text,link
0,Время подумать,15.03.2023 05:21,«Мы на месяц закрываем амурские дороги для бол...,https://www.zvportal.ru/index.php/2010-11-20-0...
1,Бюджет увеличен,10.03.2023 01:17,3 марта на очередной сессии Зейского районного...,https://www.zvportal.ru/index.php/2010-11-20-0...
2,Мусор не пройдёт!,17.03.2023 06:46,Региональный оператор города Зеи и Зейского ра...,https://www.zvportal.ru/index.php/2010-11-20-0...
